In [ ]:
!pip install langchain
!pip install sentence_transformers
!pip install pdf2image
!pip install pypdf
!pip install xformers
!pip install bitsandbytes accelerate transformers

In [ ]:
!pip install -U langchain-community

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoModelForCausalLM
from transformers import pipeline
import os
import sys

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os


In [ ]:
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA

In [ ]:
!pip install chromadb

Loading PDFs

In [ ]:
loader = PyPDFDirectoryLoader("/kaggle/input/operating")
data = loader.load()

In [ ]:
len(data)

In [ ]:
data

In [ ]:
data[0]

In [ ]:
print(data)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
docs = text_splitter.split_documents(data)
print(len(docs))

In [ ]:
docs[3]

**------------------------------------------------------------**

**Embedding download**

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
from huggingface_hub import notebook_login
import torch

In [ ]:
notebook_login()

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              #load_in_8bit=True,
                                              load_in_4bit=True
                                             )

In [ ]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.1})

In [ ]:
llm

**ChromaBD**

In [ ]:
!pip install pysqlite3-binary

In [ ]:
import pysqlite3
sys.modules['sqlite3'] = pysqlite3

In [ ]:
import sqlite3

In [ ]:
import chromadb

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
vectordb=Chroma.from_documents(docs,embedding=embeddings)

In [ ]:
vectordb.persist()

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [ ]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [ ]:
pdf_qa=ConversationalRetrievalChain.from_llm(llm=llm,
                                             retriever=vectordb.as_retriever(search_kwargs={'k':6}),
                                             verbose=False, memory=memory)

In [ ]:
result=pdf_qa({"question":"What is multithreading?"})

In [ ]:
result

In [ ]:
result['question']

In [ ]:
result['chat_history'][1]

In [ ]:
result['answer']

In [ ]:
while True:
    queryinput=input(f"prompt:")
    if queryinput=='exit':
        print('Exiting')
        sys.exit()
    if queryinput=='':
        continue
    result=pdf_qa({'question':queryinput})
    print(f"Answer:{result['answer']}")